In [3]:
import os
os.environ['TRANSFORMERS_CACHE'] = "D:/transformer_cache/"
os.environ['HF_DATASETS_CACHE'] = "D:/transformer_cache/"

# Greedy Search Decoding

#### Importing GPT-2 model

In [4]:
# hide_output
import torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel
#device = "cuda" if torch.cuda.is_available() else "cpu"
device="cpu"
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

OSError: Can't load tokenizer for 'gpt2-xl'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2-xl' is the correct path to a directory containing all relevant files for a GPT2Tokenizer tokenizer.

In [ ]:
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
# model = GPT2Model.from_pretrained('gpt2-xl')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

#### Implementing Greedy Search Decoding by selecting the token with highest probability next. The choices and their corresponding probabilities at each time step are provided in the output.

In [ ]:
# hide_output
import pandas as pd

input_txt = "What is the opposite of front?"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        #print(output)
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
        
pd.DataFrame(iterations)

#### Use the below cell to specify input_txt and generate text based on the input_txt

In [46]:
# hide_output
import pandas as pd

input_txt = "who is the prime minister of india?"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 30
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        print(iteration)
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        #print(output)
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        #print(next_token_probs.shape)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        
        # Store tokens with highest probabilities
        
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

#print(iteration)
#pd.DataFrame(iterations)

{'Input': 'who is the prime minister of india?'}
{'Input': 'who is the prime minister of india?\n'}
{'Input': 'who is the prime minister of india?\n\n'}
{'Input': 'who is the prime minister of india?\n\nThe'}
{'Input': 'who is the prime minister of india?\n\nThe answer'}
{'Input': 'who is the prime minister of india?\n\nThe answer is'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is the'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is the prime'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is the prime minister'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is the prime minister of'}
{'Input': 'who is the prime minister of india?\n\nThe answer is that he is th

In [2]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

NameError: name 'tokenizer' is not defined

#### We specify the maximum sequence length of tokens to generate in max_length. And perform greedy decoding but using the inbuilt model.generate()

In [34]:
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, 
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


#### We notice that there is repetition in generated sentences.

# Beam Search Decoding

#### We compare the decoding strategies using the log probability of the entire sentence. Higher the log probability, higher is the probability of the generated sentence based on the input provided. Hence, a higher score implies a better decoding strategy.

### Greedy Search

In [35]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label
     

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()
     

logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able

log-prob: -87.43


### Beam search with 5 beams

In [36]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery of the unicorns was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.


The scientists were conducting a study of the Andes Mountains when they discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English

log-prob: -55.23


##### We notice that beam search improves the log prob score. However, it is more time consuming.

### Beam search with #beams = 5 and no_repeat_ngram_size=2
no_repeat_ngram size sets the next token probability to 0 if it causes a repeation of no_repeat_ngram_size ngrams. It is used to avoid repeatitions.

In [37]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The discovery was made by a team of scientists from the University of California, Santa Cruz, and the National Geographic Society.

According to a press release, the scientists were conducting a survey of the area when they came across the herd. They were surprised to find that they were able to converse with the animals in English, even though they had never seen a unicorn in person before. The researchers were

log-prob: -93.12


#### We notice a reduction in log prob score when using no repeat ngram. However, we avoid repetitions. Thus, no repeat ngram feature allows us to apply a trade off between high probability tokens and repetitions

# Sampling Methods

In [1]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True,
temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

NameError: name 'model' is not defined